In [1]:
# coding: utf-8
import pandas as pd
from reportlab.pdfgen import canvas
import qrcode
import uuid
import datetime
from tkinter.filedialog import askopenfile
from tkinter.filedialog import askdirectory
import tkinter as tk
import getpass
import socket


class Common():
    def __init__(self):
        self.dt = datetime.datetime.now()
        self.uuid = uuid.uuid4()
        self.user = getpass.getuser()
        self.device = socket.gethostname()


def main():
    # GET DATA
    root = tk.Tk()
    root.withdraw()
    def wolford_data():
        #file = r"C:\Users\mkreidler\Desktop\wolford_barcodes.xlsx"
        file = askopenfile(initialdir = "C:/Users/mkreidler.BVTCCLOUD/Desktop",
                           title = "Select Excel Data File",
                           filetypes = (("Excel Files","*.xlsx"),("all files","*.*"))).name
        print(file)
        df = pd.read_excel(file)
        cols = ["swing_drop", "floor", "name"]
        df.name = df.name.apply(str)
        df.floor = df.floor.apply(str)
        df = df.sort_values(["swing_drop", "floor", "name"])
        df["cc"] = df.swing_drop + "-" + df.floor + "-" + df.name
        return df.cc.tolist()

    # Split by pages, rows, columns for 3 by 8 sticker sheets
    def sort_data(data):
        stuff = data
        df = pd.DataFrame({'stuff': stuff})
        df["num"] = df.index + 1
        df["c"] = df.num % 3
        df.c = df.c.replace(0, 3)
        df["page"] = ((df.num + 23) // 24)
        df["r"] = (((df.num - 1) // 3) + 1) % 8
        df.r = df.r.replace(0, 8)
        return df

    df = sort_data(wolford_data())


    # FILE LOCATIONS  ### NEED TO MAKE THIS A DIALOG BOX

    #logo_path = askopenfile().name
    #logo_path = r"C:\Users\mkreidler\Desktop\pdf print barcodes\BVTC-Logo-BLACK-horizontal.png"
    logo_path = r"C:\Users\mkreidler.BVTCCLOUD\Desktop\BVTC-Logo-BLACK-horizontal.png"
    #pdf_suffix = r"C:\Users\mkreidler\Desktop\pdf print barcodes\\"
    pdf_prefix = askdirectory(initialdir = "C:/Users/mkreidler.BVTCCLOUD/Desktop",
                              title = "Select folder to save files to...")
    dt = str(datetime.datetime.now()).replace(":", "_")
    ext = ".pdf"
    pdf_path = f"{pdf_prefix}\\{dt}{ext}"

    # MAKE CANVAS
    c = canvas.Canvas(pdf_path)
    c.setPageSize((612, 792))
    last_page = None

    # SPLIT INTO PAGES, COLUMNS, ROWS
    for index, row in df.iterrows():
        page = row["page"]
        if page != last_page:
            if last_page == None:
                last_page = page

            else:
                c.showPage()
                last_page = page

        col = row["c"]
        _row = row["r"]
        i = row["stuff"]
        code = f"{i}"
        print(f"page {page} row {_row} column {col} info {i}")

        # DRAW QR
        def draw_qr(data, base=pdf_prefix):
            # draw QR
            dt = str(datetime.datetime.now()).replace(":", "_")
            img_path = f"{base}\\{dt}{i}.png"
            qr = qrcode.QRCode()
            qr.add_data(data)
            img = qr.make_image().save(img_path)
            return img_path


        # DRAW 1 TAG
        def draw_tag(x_num, y_num, code):
            flip_y = {1: 8,
                      2: 7,
                      3: 6,
                      4: 5,
                      5: 4,
                      6: 3,
                      7: 2,
                      8: 1}

            y_num = flip_y[y_num]

            qr = draw_qr(code)
            x_increment = 204
            y_increment = 99

            x = x_increment * (x_num - 1)
            y = y_increment * (y_num - 1)
            com = Common()
            #c.drawString(102.5491+x,10+y,str(com.dt))

            c.setFontSize(16)
            c.drawString(102.5491 + x, 33.7174 + y, code)

            c.setFontSize(5)
            c.drawString(102.5491+x, 14+y, str(com.dt))

            c.drawInlineImage(qr, 0 + x, 0 + y, width=99, height=99)
            c.drawInlineImage(logo_path, 102.5491 + x, 71.2396 + y, width=84.2926,
                              height=14.0488)  # replace maroon with bnw later


        draw_tag(col, _row, code)

    # SAVE DOCUMENT
    c.save()
    print()
    print("Done!")


if __name__ == "__main__":
    main()




C:/Users/mkreidler/Desktop/wolford_barcodes.xlsx
page 1 row 1 column 1 info E1-1-14


OSError: Cannot open resource "C:\Users\mkreidler.BVTCCLOUD\Desktop\BVTC-Logo-BLACK-horizontal.png"
fileName='C:\\Users\\mkreidler.BVTCCLOUD\\Desktop\\BVTC-Logo-BLACK-horizontal.png' identity=[ImageReader@0xab8f550 filename='C:\\Users\\mkreidler.BVTCCLOUD\\Desktop\\BVTC-Logo-BLACK-horizontal.png']

In [ ]:
import pandas as pd

def wolford_data():
    #file = r"C:\Users\mkreidler\Desktop\wolford_barcodes.xlsx"
    file = r"C:\Users\mkreidler.BVTCCLOUD\Desktop\wolford_barcodes.xlsx"
    print(file)
    df = pd.read_excel(file)
    cols = ["swing_drop", "floor", "name"]
    df.name = df.name.apply(str)
    df.floor = df.floor.apply(str)
    df = df.sort_values(["swing_drop", "floor", "name"])
    df["cc"] = df.swing_drop + "-" + df.floor + "-" + df.name
    return df

dd = wolford_data().name.unique().tolist()
dd.sort()
dd